In [102]:
import pandas as pd
import openpyxl
import re
from datetime import datetime
from pathlib import Path
from azure.storage.blob import BlobServiceClient
import numpy as np

In [119]:
# Function to get weeks and comments where value is 1
def get_weeks_and_comments(row):
    weeks = []
    comments = []
    row_idx = int(row.name.strip("N°"))
    col_idx = 1
    for col, value in row.items():
        if value == 1:
            weeks.append(col)
            cell = sheet[row_idx + 1][col_idx]
            comments.append(cell.comment.text if cell.comment else "")
        col_idx += 1
    return pd.Series({"weeks": weeks, "comments": comments})


def idx_to_pool_number(df):
    df = (
        df.reset_index()
        .rename(columns={"index": "pool_number"})
        .assign(pool_number=lambda x: x["pool_number"].str.strip("N°"))
    )
    return df


# Function to extract information from comments
def extract_info(comment):
    equipo_match = re.search(r"Equipo:\s*(\d+)", comment)
    ns_match = re.search(r"NS:\s*(#\w*-?\w*)", comment)

    equipo = equipo_match.group(1) if equipo_match else None
    ns = ns_match.group(1) if ns_match else None

    return equipo, ns


files = [p for p in Path("/home/cecilvega/Public/pool-files").rglob("*.xlsx")]


frames = []
for file in files:

    # Read the Excel file
    excel_file = file.__str__()  # Replace with your actual file name
    wb = openpyxl.load_workbook(excel_file, data_only=True)
    sheet = wb.active

    # Create a DataFrame from the Excel data
    data = []
    for row in sheet.iter_rows(min_row=2, values_only=True):
        data.append(row)

    df = pd.DataFrame(data, columns=[cell.value for cell in sheet[1]])
    df.set_index(df.columns[0], inplace=True)

    # Process the dataframe
    df = (
        df.apply(get_weeks_and_comments, axis=1)
        .explode(["weeks", "comments"])
        .rename(columns={"weeks": "cc_week"})
        .pipe(idx_to_pool_number)
    )

    # Extract information from comments
    df[["equipo", "component_serial"]] = (
        df["comments"]
        .apply(lambda x: pd.Series(extract_info(x)))
        .reset_index(drop=True)
    )
    df = (
        df.drop(columns=["comments"])
        .assign(
            component_tag=file.stem.split("-")[-1],
            year=df["cc_week"].str.split("-").map(lambda x: x[0]),
            week=df["cc_week"].str.split("-W").map(lambda x: x[-1]).str.zfill(2),
        )
        .assign(
            cc_week=lambda x: x["year"].str.cat(
                x["week"],
                sep="-W",
            ),
            componente=lambda x: x["component_tag"].map(
                lambda x: {
                    "blower": "Blower",
                    "cd": "Cilindro Dirección",
                    "st": "Suspensión Trasera",
                    "cms": "CMSD",
                    "mt": "Motor Tracción",
                    "cl": "Cilindro Levante",
                    "mp": "Módulo Potencia",
                }[x]
            ),
        )
    )

    frames.append(df)
pool_df = pd.concat(frames)
pool_df

,pool_number,cc_week,equipo,component_serial,component_tag,year,week,componente
0,1,2023-W30,872,#178,cl,2023,30,Cilindro Levante
1,1,2024-W22,399,#LM017275,cl,2024,22,Cilindro Levante
2,2,2023-W35,861,#RLTC707,cl,2023,35,Cilindro Levante
3,2,2024-W22,399,#223,cl,2024,22,Cilindro Levante
4,3,2023-W41,856,#101,cl,2023,41,Cilindro Levante
...,...,...,...,...,...,...,...,...
31,14,2024-W21,857,#EMWK308-1,cd,2024,21,Cilindro Dirección
32,15,2023-W46,848,#SW-CD73,cd,2023,46,Cilindro Dirección
33,15,2024-W22,867,#463,cd,2024,22,Cilindro Dirección
34,16,2023-W51,874,#63LB30,cd,2023,51,Cilindro Dirección


In [120]:
comp_df = (
    pd.read_excel(
        "/home/cecilvega/Public/PLANILLA DE CONTROL CAMBIO DE COMPONENTES.xlsx"
    )
    .dropna(subset=["COMPONENTE"])
    .rename(
        columns={
            "EQUIPO": "equipo",
            "COMPONENTE": "componente",
            "N/S RETIRADO": "component_serial",
            "W": "cc_week",
        }
    )
).assign(
    equipo=lambda x: x["equipo"].str.extract(r"(\d+)"),
    cc_week=lambda x: x["FECHA DE CAMBIO"]
    .dt.year.astype(str)
    .str.cat(x["cc_week"].astype(str).str.zfill(2), sep="-W"),
)
comp_df

,equipo,componente,SUB COMPONENTE,MODELO,POSICION,FECHA DE CAMBIO,cc_week,HORA EQ,HORA CC,MENOR AL 90% TBO,...,OS 14.000,RMCARE,Pauta puesta servicio(entrante),TSI,No Conformidad,CLASIFICACIÓN,COMENTARIO ENTRANTE,COMENTARIO SALIENTE,Proyeccion Horómetro,% uso Proyectado
0,862,Motor Tracción,MOTOR TRACCIÓN,960E,IZQUIERDO,2019-01-01,2019-W01,25516,14025,0.0,...,NaN,CERRADA,NaN,NaN,NaN,NaN,NaN,NaN,34391.0,1.432958
1,862,Motor Tracción,Motor Eléctrico,960E,IZQUIERDO,2019-01-01,2019-W01,25516,14025,0.0,...,NaN,CERRADA,NaN,NaN,NaN,NaN,NaN,NaN,34391.0,1.432958
2,862,CMSD,Freno Servicio,960E,IZQUIERDO,2019-01-01,2019-W01,25516,14025,0.0,...,NaN,CERRADA,NaN,NaN,NaN,NaN,NaN,NaN,34391.0,1.432958
3,862,Motor Tracción,Freno Estacionamiento,960E,IZQUIERDO,2019-01-01,2019-W01,25516,14025,0.0,...,NaN,CERRADA,NaN,NaN,NaN,NaN,NaN,NaN,34391.0,1.432958
4,860,Blower,Blower,960E,IZQUIERDO,2019-01-01,2019-W01,26904,26007,1.0,...,NaN,CERRADA,NaN,NaN,NaN,NaN,NaN,NaN,34391.0,1.719550
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2735,320,Blower,Blower,960E,IZQUIERDO,2024-07-15,2024-W29,87199,13291,0.0,...,NaN,ABIERTO,NaN,NaN,NaN,Producto,COMPONENTE CLIENTE,NaN,17.0,0.000850
2736,872,Motor Tracción,MOTOR TRACCIÓN,960E,IZQUIERDO,2024-07-15,2024-W29,58846,27730,1.0,...,NaN,ABIERTO,NaN,NaN,NaN,TBO,RETROFIT,NORMAL,17.0,0.000708
2737,872,Motor Tracción,Motor Eléctrico,960E,IZQUIERDO,2024-07-15,2024-W29,58846,27730,1.0,...,NaN,ABIERTO,NaN,NaN,NaN,TBO,RETROFIT,NORMAL,17.0,0.000708
2738,872,Motor Tracción,Freno Servicio,960E,IZQUIERDO,2024-07-15,2024-W29,58846,27730,1.0,...,NaN,ABIERTO,NaN,NaN,NaN,TBO,RETROFIT,NORMAL,17.0,0.000708


In [128]:
df = pd.merge(
    pool_df,
    comp_df,
    on=["equipo", "componente", "component_serial", "cc_week"],
    how="left",
)
df = df.assign(pool_type=np.where(df["HORA CC"] > 0.9 * df["TBO"], "P", "I"))
df = df.assign(
    ohv_normal=df["componente"].map(
        lambda x: {
            "Blower": 51,
            "Cilindro Dirección": 46,
            "Suspensión Trasera": 65,
            "CMSD": 64,
            "Motor Tracción": 74,
            "Cilindro Levante": 75,
            "Módulo Potencia": 64,
        }[x]
    ),
    ohv_unplanned=df["componente"].map(
        lambda x: {
            "Blower": 101,
            "Cilindro Dirección": 96,
            "Suspensión Trasera": 125,
            "CMSD": 124,
            "Motor Tracción": 134,
            "Cilindro Levante": 135,
            "Módulo Potencia": 114,
        }[x]
    ),
)
df.to_csv("pool-consolidated.csv", index=False)

In [116]:
df

,pool_number,cc_week,equipo,component_serial,component_tag,year,week,componente,SUB COMPONENTE_x,MODELO_x,...,OS 14.000,RMCARE,Pauta puesta servicio(entrante),TSI,No Conformidad,CLASIFICACIÓN,COMENTARIO ENTRANTE,COMENTARIO SALIENTE,Proyeccion Horómetro,% uso Proyectado
0,1,2023-W30,872,#178,cl,2023,30,Cilindro Levante,CILINDRO LEVANTE,960E,...,NaN,CERRADA,NaN,NaN,NaN,TBO,NaN,NaN,5984.0,0.230154
1,1,2024-W22,399,#LM017275,cl,2024,22,Cilindro Levante,CILINDRO LEVANTE,960E,...,NaN,CERRADO,NaN,NaN,NaN,Producto/TBO,NaN,NaN,850.0,0.035417
2,2,2023-W35,861,#RLTC707,cl,2023,35,Cilindro Levante,CILINDRO LEVANTE,960E,...,NaN,CERRADA,NaN,NaN,NaN,Producto,NaN,NaN,5389.0,0.207269
3,2,2024-W22,399,#223,cl,2024,22,Cilindro Levante,CILINDRO LEVANTE,960E,...,NaN,CERRADO,NaN,NaN,NaN,Producto,NaN,NaN,765.0,0.031875
4,3,2023-W41,856,#101,cl,2023,41,Cilindro Levante,CILINDRO LEVANTE,960E,...,NaN,CERRADO,NaN,NaN,NaN,TBO,NaN,NaN,4709.0,0.181115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,14,2024-W21,857,#EMWK308-1,cd,2024,21,Cilindro Dirección,Cilindro Dirección,960E,...,NaN,NaN,NaN,NaN,NaN,Producto,NaN,NaN,867.0,0.072250
192,15,2023-W46,848,#SW-CD73,cd,2023,46,Cilindro Dirección,Cilindro Dirección,960E,...,NaN,CERRADO,NaN,NaN,NaN,Producto,NaN,CILINDRO SALIENTE ES PROPIEDAD DE CLIENTE,4131.0,0.344250
193,15,2024-W22,867,#463,cd,2024,22,Cilindro Dirección,Cilindro Dirección,960E,...,NaN,CERRADO,NaN,NaN,NaN,Producto,NaN,NaN,816.0,0.068000
194,16,2023-W51,874,#63LB30,cd,2023,51,Cilindro Dirección,Cilindro Dirección,960E,...,NaN,CERRADO,NaN,NaN,NaN,Producto,NaN,NaN,3570.0,0.297500


In [101]:
df

,pool_number,cc_week,equipo,component_serial,component_tag,year,week,componente,SUB COMPONENTE_x,MODELO_x,...,OS 14.000,RMCARE,Pauta puesta servicio(entrante),TSI,No Conformidad,CLASIFICACIÓN,COMENTARIO ENTRANTE,COMENTARIO SALIENTE,Proyeccion Horómetro,% uso Proyectado
0,1,2023-W30,872,#178,cl,2023,30,Cilindro Levante,CILINDRO LEVANTE,960E,...,NaN,CERRADA,NaN,NaN,NaN,TBO,NaN,NaN,5984.0,0.230154
1,1,2024-W22,399,#LM017275,cl,2024,22,Cilindro Levante,CILINDRO LEVANTE,960E,...,NaN,CERRADO,NaN,NaN,NaN,Producto/TBO,NaN,NaN,850.0,0.035417
2,2,2023-W35,861,#RLTC707,cl,2023,35,Cilindro Levante,CILINDRO LEVANTE,960E,...,NaN,CERRADA,NaN,NaN,NaN,Producto,NaN,NaN,5389.0,0.207269
3,2,2024-W22,399,#223,cl,2024,22,Cilindro Levante,CILINDRO LEVANTE,960E,...,NaN,CERRADO,NaN,NaN,NaN,Producto,NaN,NaN,765.0,0.031875
4,3,2023-W41,856,#101,cl,2023,41,Cilindro Levante,CILINDRO LEVANTE,960E,...,NaN,CERRADO,NaN,NaN,NaN,TBO,NaN,NaN,4709.0,0.181115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,14,2024-W21,857,#EMWK308-1,cd,2024,21,Cilindro Dirección,Cilindro Dirección,960E,...,NaN,NaN,NaN,NaN,NaN,Producto,NaN,NaN,867.0,0.072250
192,15,2023-W46,848,#SW-CD73,cd,2023,46,Cilindro Dirección,Cilindro Dirección,960E,...,NaN,CERRADO,NaN,NaN,NaN,Producto,NaN,CILINDRO SALIENTE ES PROPIEDAD DE CLIENTE,4131.0,0.344250
193,15,2024-W22,867,#463,cd,2024,22,Cilindro Dirección,Cilindro Dirección,960E,...,NaN,CERRADO,NaN,NaN,NaN,Producto,NaN,NaN,816.0,0.068000
194,16,2023-W51,874,#63LB30,cd,2023,51,Cilindro Dirección,Cilindro Dirección,960E,...,NaN,CERRADO,NaN,NaN,NaN,Producto,NaN,NaN,3570.0,0.297500


In [97]:
df.query("TBO.isnull()")

UndefinedVariableError: name 'TBO' is not defined

In [13]:
df

,pool_number,repair_start_week,equipo,component_serial,repair_start_date,component_tag,componente
0,1,2023-W30,872,#178,2023-07-24,cl,Cilindro Levante
1,1,2024-W22,399,#LM017275,2024-05-27,cl,Cilindro Levante
2,2,2023-W35,861,#RLTC707,2023-08-28,cl,Cilindro Levante
3,2,2024-W22,399,#223,2024-05-27,cl,Cilindro Levante
4,3,2023-W41,856,#101,2023-10-09,cl,Cilindro Levante
5,4,2023-W51,398,#LM017653,2023-12-18,cl,Cilindro Levante
6,5,2024-W2,883,#223,2024-01-08,cl,Cilindro Levante
7,6,2024-W2,883,#222,2024-01-08,cl,Cilindro Levante
8,7,2024-W8,856,#LM021235,2024-02-19,cl,Cilindro Levante
9,8,2024-W8,870,#180,2024-02-19,cl,Cilindro Levante


In [13]:
110 / 7

15.714285714285714

In [8]:
files

[PosixPath('/home/cecilvega/Public/pool-files/pool-cc-cl.xlsx')]

In [ ]:
df

In [ ]:
"/home/cecilvega/Public"

In [3]:
account_url = "https://tecnologiakchqa.blob.core.windows.net"
sas_token = "?sv=2023-01-03&spr=https%2Chttp&st=2024-07-15T12%3A00%3A00Z&se=2028-01-01T16%3A44%3A00Z&sr=c&sp=racwdxltf&sig=AKC%2Fcx8Rfxma4l38WBC6f%2BkLRAHuMjjN2x7jrhjqVsg%3D"
prefix = "FAENAS/ESCONDIDA/y=2023/m=09/MEL_Septiembre_2023/TK848"
container_name = "kch-ddmm"
blob_service_client = BlobServiceClient(account_url=account_url, credential=sas_token)

In [25]:
container_client = blob_service_client.get_container_client(container_name)
blob_list = container_client.list_blobs(name_starts_with=prefix)
blob_list = [f.name for f in blob_list]

['FAENAS/ESCONDIDA/y=2023/m=09/MEL_Septiembre_2023/TK848/pool-files/.~lock.pool-cc-cms.xlsx#',
 'FAENAS/ESCONDIDA/y=2023/m=09/MEL_Septiembre_2023/TK848/pool-files/pool-cc-blower.xlsx',
 'FAENAS/ESCONDIDA/y=2023/m=09/MEL_Septiembre_2023/TK848/pool-files/pool-cc-cd.xlsx',
 'FAENAS/ESCONDIDA/y=2023/m=09/MEL_Septiembre_2023/TK848/pool-files/pool-cc-cl.xlsx',
 'FAENAS/ESCONDIDA/y=2023/m=09/MEL_Septiembre_2023/TK848/pool-files/pool-cc-cms.xlsx',
 'FAENAS/ESCONDIDA/y=2023/m=09/MEL_Septiembre_2023/TK848/pool-files/pool-cc-mp.xlsx',
 'FAENAS/ESCONDIDA/y=2023/m=09/MEL_Septiembre_2023/TK848/pool-files/pool-cc-mt.xlsx',
 'FAENAS/ESCONDIDA/y=2023/m=09/MEL_Septiembre_2023/TK848/pool-files/pool-cc-st.xlsx']

In [ ]:
def process_excel_files(connection_string: str, container_name: str, folder_name: str):
    blob_service_client = BlobServiceClient.from_connection_string(connection_string)
    container_client = blob_service_client.get_container_client(container_name)

    # List all blobs in the specified folder
    blob_list = container_client.list_blobs(name_starts_with=folder_name)

    for blob in blob_list:
        if blob.name.endswith(".xlsx"):
            print(f"Processing file: {blob.name}")

            # Download the blob content
            blob_client = container_client.get_blob_client(blob.name)
            download_stream = blob_client.download_blob()
            content = download_stream.readall()

            # Load the Excel file
            workbook = openpyxl.load_workbook(BytesIO(content), data_only=True)
            sheet = workbook.active

            # Process the Excel file (example: print the value of cell A1)
            print(f"Value in A1: {sheet['A1'].value}")

            # Close the workbook
            workbook.close()

# Función antigua para abrir archivos

In [9]:
# Function to get weeks and comments where value is 1
def get_weeks_and_comments(row):
    weeks = []
    comments = []
    row_idx = int(row.name.strip("N°"))
    col_idx = 1
    for col, value in row.items():
        if value == 1:
            weeks.append(col)
            cell = sheet[row_idx + 1][col_idx]
            comments.append(cell.comment.text if cell.comment else "")
        col_idx += 1
    return pd.Series({"weeks": weeks, "comments": comments})


def get_end_week(row):
    weeks = []
    pool_repair_types = []
    row_idx = int(row.name.strip("N°"))
    for col_idx in range(0, row.__len__()):
        if col_idx > 2:
            prev_cell = sheet[row_idx + 1][col_idx - 1]
            cell = sheet[row_idx + 1][col_idx]
            if (
                (prev_cell.fill.fgColor.rgb != cell.fill.fgColor.rgb)
                & (prev_cell.value is None)
                & (prev_cell.fill.fgColor.rgb in ["FFEDBFBB", "FFC5E0B4", "FFE88880"])
                # & (prev_cell.fill.fgColor.rgb in ["FFC5E0B4", "FFE88880"])
            ):
                weeks.append(list(row.keys())[col_idx - 2])
                if prev_cell.fill.fgColor.rgb in ["FFEDBFBB", '"FFE88880"']:
                    pool_repair_types.append("I")
                else:
                    pool_repair_types.append("P")

    return pd.Series({"weeks": weeks, "pool_repair_type": pool_repair_types})


def idx_to_pool_number(df):
    df = (
        df.reset_index()
        .rename(columns={"index": "pool_number"})
        .assign(pool_number=lambda x: x["pool_number"].str.strip("N°"))
    )
    return df


# Function to extract information from comments
def extract_info(comment):
    equipo_match = re.search(r"Equipo:\s*(\d+)", comment)
    ns_match = re.search(r"NS:\s*(#\w*-?\w*)", comment)

    equipo = equipo_match.group(1) if equipo_match else None
    ns = ns_match.group(1) if ns_match else None

    return equipo, ns


files = [p for p in Path("/home/cecilvega/Public/pool-files").rglob("*cl.xlsx")]


frames = []
for file in files:

    # Read the Excel file
    excel_file = file.__str__()  # Replace with your actual file name
    wb = openpyxl.load_workbook(excel_file, data_only=True)
    sheet = wb.active

    # Create a DataFrame from the Excel data
    data = []
    for row in sheet.iter_rows(min_row=2, values_only=True):
        data.append(row)

    df = pd.DataFrame(data, columns=[cell.value for cell in sheet[1]])
    df.set_index(df.columns[0], inplace=True)

    # Process the dataframe
    start_repair_df = (
        df.apply(get_weeks_and_comments, axis=1)
        .explode(["weeks", "comments"])
        .rename(columns={"weeks": "repair_start_week"})
        .pipe(idx_to_pool_number)
    )

    # Extract information from comments
    start_repair_df[["equipo", "ns"]] = start_repair_df["comments"].apply(
        lambda x: pd.Series(extract_info(x))
    )
    start_repair_df = start_repair_df.drop(columns=["comments"]).assign(
        repair_start_date=start_repair_df["repair_start_week"].map(
            lambda x: datetime.strptime(x + "-1", "%Y-W%W-%w")
        )
    )
    end_repair_df = (
        df.apply(get_end_week, axis=1)
        .explode(["weeks", "pool_repair_type"])
        .rename(columns={"weeks": "repair_end_week"})
        .pipe(idx_to_pool_number)
    )
    end_repair_df = end_repair_df.assign(
        repair_end_date=end_repair_df["repair_end_week"].map(
            lambda x: datetime.strptime(x + "-1", "%Y-W%W-%w")
        )
    )

    df = pd.merge_asof(
        start_repair_df.sort_values("repair_start_date"),
        end_repair_df.sort_values("repair_end_date"),
        by="pool_number",
        left_on="repair_start_date",
        right_on="repair_end_date",
        direction="forward",
    ).assign(component=file.stem.split("-")[-1])
    frames.append(df)
df = pd.concat(frames)